In [1]:
import math
from scipy.optimize import minimize
import numpy as np

In [2]:
### tools：

def minfunction(f, x0, bounds):
    """
    求一个函数在边界内的最小值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    res = minimize(f, x0, method='SLSQP', bounds=bounds)
    return res

def maxfunction(f, x0, bounds):
    """
    求一个函数在边界内的最大值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    fun = lambda X : -f(X)
    res = minimize(fun, x0, method='SLSQP', bounds=bounds)
    return res


In [3]:
### 目标函数：

bounds = [(100, 10000), (1000, 10000), (1000, 10000), (10, 1000), (10, 1000), (10, 1000), (10, 1000), (10, 1000)]
x_opt = [579.306685017979589, 1359.97067807935605, 5109.97065743133317, 182.01769963061534, 295.601173702746792,
         217.982300369384632, 286.41652592786852, 395.601173702746735]
y_opt = 7049.24802052867


def G10(X):
    y = X[0] + X[1] + X[2]

    g1 = -1 + 0.0025 * (X[3] + X[5])
    g2 = -1 + 0.0025 * (X[4] + X[6] - X[3])
    g3 = -1 + 0.01 * (X[7] - X[4])
    g4 = -X[0] * X[5] + 833.33252 * X[3] + 100 * X[0] - 83333.333
    g5 = -X[1] * X[6] + 1250 * X[4] + X[1] * X[3] - 1250 * X[3]
    g6 = -X[2] * X[7] + 1250000 + X[2] * X[4] - 2500 * X[4]

    return y, g1, g2, g3, g4, g5, g6

In [4]:
# g1 = lambda X: G10(X)[1]
# g2 = lambda X: G10(X)[2]
# g3 = lambda X: G10(X)[3]
# g4 = lambda X: G10(X)[4]
# g5 = lambda X: G10(X)[5]
# g6 = lambda X: G10(X)[6]
# x0 = [100,1000,10,10,10,10000,10,10]

In [5]:
# maxfunction(g6,x0,bounds)

In [6]:
### 适应度函数
def fitnessG10_num(X, epsion):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = 1220.174
    fmin = 0.008509619375000003
    count = np.sum(np.array(problem1(X))[1:5]>0)
    
    return -(problem1(X)[0]+epsion*(fmax-fmin)*count)  

def fitnessG10_vio(X):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = 30000
    fmin = 2100
    vmax = 4112515.88
    e = 0.000001
    n = 6
    
    res = G10(X)
    fitness = -res[0]
    count = np.sum(np.array(res)[1:7]>0)
    
    if count > 0:
        for i in range(1,len(res)):
            if res[i] > 0:
                cathy = (fmax - fmin)*(e*vmax*n/count+res[i])/(e*vmax)
                fitness = fitness - cathy
    return fitness   


In [7]:
### PSO
class PSO_fit():
    def __init__(self, pN, dim, max_iter, bounds):
        self.wmax = 0.8 #最大的惯性权重
        self.wmin = 0.4 #最小的惯性权重
        self.c1 = 2  #加速因子
        self.c2 = 2  #加速因子
        self.pN = pN  # 粒子数量
        self.dim = dim  # 搜索维度
        self.max_iter = max_iter  # 迭代次数
        self.bounds = bounds
        
        self.X = np.zeros((self.pN, self.dim))  # 所有粒子的位置：一个pN*dim的矩阵
        self.V = np.zeros((self.pN, self.dim))  # 左右粒子的速度：一个pN*dim的矩阵
        self.pbest = np.zeros((self.pN, self.dim))  # 个体经历的最佳位置：一个pN*dim的矩阵
        self.gbest = np.zeros((self.pN, self.dim))      # 全局最佳位置：一个pN*dim的矩阵
        self.p_fit = np.full(self.pN,-100000000.0)  # 每个个体的历史最佳适应值
        self.g_fit = np.full(self.pN,-100000000.0)    # 全局最佳适应值
    
    def init_Population(self):
        """
        初始化种群
        :return:
        """
        
        for i in range(self.pN):
            for j in range(self.dim):
                self.X[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
                self.V[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
        for i in range(self.pN):
            tmp = fitnessG10_vio(self.X[i])
            if (tmp > self.p_fit[i]):
                self.p_fit[i] = tmp
                self.pbest[i] = self.X[i]
                
        for i in range(self.pN):
            indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/100))
            for j in indexes:
                if self.p_fit[j] > self.g_fit[i]:
                    self.g_fit[i] = self.p_fit[j]
                    self.gbest[i] = self.pbest[j]

#             self.pbest[i] = self.X[i]
#             tmp = fitnessG01_num(self.X[i])
#             self.p_fit[i] = tmp
#             if (tmp > self.fit):
#                 self.fit = tmp
#                 self.gbest = self.X[i]
        
    def iterator(self):
        """
        迭代
        """
        for t in range(self.max_iter):
            w = self.wmin + (self.wmax - self.wmin) * np.exp(- (4 * t / self.max_iter) * (4 * t / self.max_iter))
            
            for i in range(self.pN):
                self.V[i] = w*self.V[i]+self.c1*np.random.random()*(self.pbest[i]-self.X[i])+self.c2*np.random.random()*(self.gbest[i]-self.X[i])
                self.X[i] = self.X[i] + self.V[i]
                
                """
                控制在搜索范围内
                """ 
                for j in range(self.dim):
                    if self.X[i][j] < self.bounds[j][0]:
                        self.X[i][j] = self.bounds[j][0]
                    if self.X[i][j] > self.bounds[j][1]:
                        self.X[i][j] = self.bounds[j][1]     
            for i in range(self.pN):
                tmp = fitnessG10_vio(self.X[i])
                if (tmp > self.p_fit[i]):
                    self.p_fit[i] = tmp
                    self.pbest[i] = self.X[i]
            for i in range(self.pN):
                indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/100))
                for j in indexes:
                    if self.p_fit[j] > self.g_fit[i]:
                        self.g_fit[i] = self.p_fit[j]
                        self.gbest[i] = self.pbest[j]
            print(self.g_fit)  
        return self.gbest, self.g_fit

In [8]:
PSO = PSO_fit(pN=4000, dim=8, max_iter=500, bounds=bounds)

In [9]:
PSO.init_Population()

In [10]:
gbest, gfit = PSO.iterator()

[-194452.44957131 -197910.55702204 -196544.55319224 ... -197882.27687189
 -190599.11705993 -194452.44957131]
[-189088.21839799 -188013.83761717 -188971.43994432 ... -188013.83761717
 -190599.11705993 -193162.09438577]
[-189088.21839799 -188013.83761717 -188971.43994432 ... -188013.83761717
 -187367.43851582 -191697.09974596]
[-183835.16697328 -184768.70976308 -183420.61337812 ... -182139.10217708
 -185942.90155513 -190598.52270693]
[-182656.92359236 -180929.93248355 -180577.10139251 ... -182139.10217708
 -185942.90155513 -180577.10139251]
[-182574.54170515 -180512.08306241 -180577.10139251 ... -182139.10217708
 -180869.27411511 -180577.10139251]
[-179535.20560365 -179546.69581826 -180577.10139251 ... -180273.3355964
 -180602.91662702 -180577.10139251]
[-179535.20560365 -179546.69581826 -180327.62316981 ... -179662.25556873
 -180060.16037625 -180577.10139251]
[ -11100.         -179546.69581826 -180327.62316981 ... -179662.25556873
 -179535.20560365 -178815.67272748]
[ -11100.         -1

[-8468.52767928 -8453.5698052  -8435.8988354  ... -8462.81059013
 -8404.56544438 -8404.56544438]
[-8468.52767928 -8404.56544438 -8435.8988354  ... -8436.12689705
 -8404.56544438 -8404.56544438]
[-8468.52767928 -8404.56544438 -8404.56544438 ... -8413.34626248
 -8404.56544438 -8404.56544438]
[-8424.29771579 -8404.56544438 -8404.56544438 ... -8399.8283637
 -8404.56544438 -8394.67910553]
[-8424.29771579 -8404.56544438 -8404.56544438 ... -8399.8283637
 -8404.56544438 -8394.67910553]
[-8424.29771579 -8404.56544438 -8404.56544438 ... -8399.8283637
 -8404.56544438 -8394.67910553]
[-8424.14299436 -8404.56544438 -8404.56544438 ... -8399.8283637
 -8404.56544438 -8394.67910553]
[-8417.19628193 -8404.56544438 -8404.56544438 ... -8384.05015912
 -8404.56544438 -8394.67910553]
[-8417.19628193 -8404.56544438 -8404.56544438 ... -8384.05015912
 -8380.51424253 -8394.67910553]
[-8417.19628193 -8404.56544438 -8404.56544438 ... -8384.05015912
 -8359.23074245 -8380.51424253]
[-8417.19628193 -8404.56544438 -84

[-8332.12161054 -8332.12555533 -8332.11938514 ... -8332.12647665
 -8332.12913978 -8332.1262728 ]
[-8332.12161054 -8332.12555533 -8332.11938514 ... -8332.12647665
 -8332.12913978 -8332.12555533]
[-8332.12161054 -8332.12145365 -8332.11938514 ... -8332.12647665
 -8332.12747957 -8332.12555533]
[-8332.12161054 -8332.11587897 -8332.11938514 ... -8332.11938514
 -8332.12161054 -8332.11432144]
[-8332.12161054 -8332.11587897 -8332.11938514 ... -8332.11938514
 -8332.11969528 -8332.11432144]
[-8332.12161054 -8332.11587897 -8332.11938514 ... -8332.11938514
 -8332.11969528 -8332.11432144]
[-8332.12161054 -8332.11587897 -8332.11938514 ... -8332.11730656
 -8332.11969528 -8332.11432144]
[-8332.11591986 -8332.11587897 -8332.11667503 ... -8332.11730656
 -8332.11969528 -8332.11432144]
[-8332.11591986 -8332.11587897 -8332.11667503 ... -8332.11719849
 -8332.11905121 -8332.11432144]
[-8332.11538451 -8332.10723282 -8332.11667503 ... -8332.11673345
 -8332.11414779 -8332.11432144]
[-8332.11538451 -8332.10723282

[-7252.33462207 -7252.21061812 -7252.33462207 ... -7252.37555583
 -7251.95443769 -7252.21979775]
[-7251.88003146 -7252.21061812 -7252.33462207 ... -7252.2786221
 -7251.95443769 -7252.21979775]
[-7251.88003146 -7252.21061812 -7252.06583401 ... -7252.05265858
 -7251.95443769 -7252.21979775]
[-7251.88003146 -7252.14985664 -7252.06583401 ... -7252.05265858
 -7251.87824262 -7252.21979775]
[-7251.78806725 -7252.11744747 -7251.92476618 ... -7251.92476618
 -7251.78344155 -7252.0496593 ]
[-7251.78806725 -7251.72160638 -7251.85446643 ... -7251.67759379
 -7251.78344155 -7252.0496593 ]
[-7251.54848462 -7251.63932512 -7251.85446643 ... -7251.67759379
 -7251.78344155 -7251.79327948]
[-7251.54848462 -7251.63932512 -7251.63183066 ... -7251.39654341
 -7251.78121179 -7251.46322447]
[-7251.54848462 -7251.63932512 -7251.63183066 ... -7251.39654341
 -7251.62335164 -7251.46322447]
[-7251.54848462 -7251.63932512 -7251.52525102 ... -7251.39654341
 -7251.45835055 -7251.46322447]
[-7251.54848462 -7251.60874894 

[-7250.966835   -7250.96683485 -7250.96688745 ... -7250.96680022
 -7250.96681721 -7250.96686676]
[-7250.96680123 -7250.96683485 -7250.96681194 ... -7250.96671027
 -7250.96681721 -7250.96686676]
[-7250.96660447 -7250.96670909 -7250.96672766 ... -7250.96671027
 -7250.96662488 -7250.9666453 ]
[-7250.96657485 -7250.96641606 -7250.96609537 ... -7250.96638088
 -7250.96654506 -7250.96645953]
[-7250.96539106 -7250.96583993 -7250.96496529 ... -7250.96585191
 -7250.96566291 -7250.96605614]
[-7250.9645812  -7250.96495779 -7250.96496529 ... -7250.96454764
 -7250.96508942 -7250.96489407]
[-7250.96290594 -7250.96391277 -7250.96316089 ... -7250.96199708
 -7250.96458814 -7250.96249569]
[-7250.96209225 -7250.96204915 -7250.96112797 ... -7250.95873738
 -7250.96111202 -7250.96030874]
[-7250.95612174 -7250.95778061 -7250.95411832 ... -7250.9585846
 -7250.95791813 -7250.95533218]
[-7250.9495352  -7250.95147552 -7250.95045353 ... -7250.94827815
 -7250.94903122 -7250.95122077]
[-7250.93996576 -7250.94102085 

[-7049.71995662 -7049.69675089 -7049.63998826 ... -7049.79170083
 -7049.72268645 -7049.74027353]
[-7049.64920956 -7049.69675089 -7049.63998826 ... -7049.77353093
 -7049.72268645 -7049.73639236]
[-7049.63293934 -7049.62356581 -7049.63998826 ... -7049.65812931
 -7049.72268645 -7049.53507648]
[-7049.56144455 -7049.62356581 -7049.60172911 ... -7049.65812931
 -7049.64749875 -7049.53507648]
[-7049.56144455 -7049.61939289 -7049.60172911 ... -7049.54991517
 -7049.51725953 -7049.53507648]
[-7049.56144455 -7049.60793686 -7049.54462144 ... -7049.47857047
 -7049.51725953 -7049.53507648]
[-7049.52890212 -7049.550759   -7049.48989498 ... -7049.47857047
 -7049.4725117  -7049.52201372]
[-7049.52890212 -7049.55052704 -7049.47806473 ... -7049.4391612
 -7049.4725117  -7049.52201372]
[-7049.50950769 -7049.50066009 -7049.47806473 ... -7049.4391612
 -7049.4725117  -7049.46743506]
[-7049.46743506 -7049.43153696 -7049.44732719 ... -7049.4391612
 -7049.4725117  -7049.43121207]
[-7049.41611512 -7049.43153696 -7

In [11]:
G10(gbest[0])

(7049.2481898876895,
 -6.545785136147231e-09,
 -2.7095802268561897e-09,
 -7.798441781226018e-09,
 -0.0035573197237681597,
 -0.002161278302082792,
 -0.001213297713547945)